# Исследование надежности заемщиков.

Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

Для получения обзора таблицы напечаем первые 10 строк,применяем метод display. Для получения информации и таблице пименем метод info()

In [1]:
import pandas as pd 
import numpy as np

In [2]:
from functools import reduce
df = pd.read_csv('/datasets/data.csv')
#display(df.head(10)) #вывели первые 5 строк для просмотра таблицы и значений
df.info() # вывели информацию по датафрейму узнали количество строк

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Обнаужены пропуски в столбце total_income(ежемесячный доход) и days_employed(общий трудовой стаж в днях).
Пропуски возникли скорее всего в результате технологической ошибки.
Чтобы не удалять нужные данные в таблице их можно заменить медианными.

In [3]:
df_pass = df.isna().sum() # посчитали общую сумму пропусков в столбцах days_employed и total_income
total_income_values = df['total_income'].isna().mean()
total_income_conversion = total_income_values/df['total_income'].count() # доля пропущенных значений для стольбца total_income
total_income_median = df['total_income'].median() # нашли медиану для total_income
df['total_income'] = df['total_income'].fillna(total_income_median) # заменили пропущенные значения на медиану

print(df.isna().sum())
#display(df.head(20))
print(f'Доля пропущенных значений в стобце total_income: {total_income_conversion:.1%}') # доля пропущенных значений столбца total_income

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64
Доля пропущенных значений в стобце total_income: 0.0%


### Шаг 2.1 Заполнение пропусков

### Шаг 2.2 Проверка данных на аномалии и исправления.

Выявленны аномалии для столбцов days_employed и education. В days_employed имеються минусовые значения, убирем их функцией abs.
Также для этого столбца заменим пропущенные значения медианными. В столбце children также имееться минусовое значение и количество детей указано 20, предположим что это ошибка человека вбивавшего данные.Уберем минусовые значения и 20 поменяем на 2.

In [4]:
df['days_employed'] = df['days_employed'].abs() #убираем отрицательные значения столбца days_employed
days_employed_conversion = df['days_employed'].isna().sum()/df['days_employed'].count() # доля пропущенных значений для столбца days_employed
days_employed_median = df['days_employed'].median() # нашли медиану для total_income
df['days_employed'] = df['days_employed'].fillna(days_employed_median) # заменили пропущенные значения на медиану
display(df.head())
days_employed_unique = df['children'].unique() # ищем уникальные значения в столбце children
df['children'] = df['children'].abs() #убираем отрицательные значения столбца children
df['children'] = df['children'].replace(20,2)
print(df['children'].unique())

print(f'Доля пропущенных значений в стобце days_employed: {days_employed_conversion:.1%}') # доля пропущенных значений столбца days_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


[1 0 3 2 4 5]
Доля пропущенных значений в стобце days_employed: 11.2%


### Шаг 2.3. Изменение типов данных.

В столбцах total_income, days_employed переведем все значения в целочисленный.

In [5]:
df = df.astype({'total_income': int})
df = df.astype({'days_employed': int})
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.4. Удаление дубликатов.

В education слова написаны с разными регистрами, приведем к одному, строчному.
Выявили дупликаты методом duplicated и методом drop_duplicates() удалил дубликаты
Дупликаты могли возникнуть из-за сбоя прогрраммы, или ошибки человека который дваждый ввел одни и теже данные

In [6]:
df['education'] = df.education.str.lower() # преобразовали все буквенные символы в строчные.
duplicated_df = df[df.duplicated()].head() # нашли все дубликаты
#display(duplicated_df) # вывели на экран дупликаты для проверки
df = df.drop_duplicates().reset_index(drop=True) # удалили дупликаты с заменой старых индексов
print(df.duplicated().sum()) # проверрили количество дупликатов
display(df.head(10))

0


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Чтобы не перегружать таблицу данными сделаем следующие действия:
Создаем новый датафрейм где каждому уникальному значению из education соответствует уникальное значение education_id
Создаем новый датафрейм где каждому уникальному значению из family_status соответствует уникальное значение family_status_id
Затем удалим столбцы  education и family_status

In [7]:
education_id = df['education'].unique() 
family_status_id = df['family_status'].unique()
#print(education_id)
#print(family_status_id)
df.drop(['education', 'family_status'], axis=1, inplace=True) # удаление столбцов education и family_status методом drop()
display(df.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,1,0,M,сотрудник,0,135823,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

Создадим новую таблицу с присвоением доходов к определенной категории. И посчитаем количество людей в каждой категории.

In [8]:
def total_income_category (row):
    if 0 <= row <= 30000:
        return 'E'
    if 30001 <= row <= 50000:
        return 'D'
    if 50001 <= row <= 200000:
        return 'C'
    if 200001 <= row <= 1000000:
        return 'B'
    return 'A'

df['total_income_category'] = df['total_income'].apply(total_income_category)
print(df['total_income_category'].value_counts())
display(df.head())

C    16016
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

Находим уникальные значения столбца purpose, и видем что там много одинаковых целей написанных по разному.
Создаем новую таблицу, чтобы также поделить на категории цели кредита.
После этого можем удалить столбец purpose и total_income.

In [9]:
#print(df['purpose'].unique()) # вывели на экрран уникальные значения в столбце purpose
def purpose_category(index): # создаем собственную функцию для нового стобца purpose_category
# используем подстроки для данных по автомобилю  
    if 'авто' in index:
        return 'операции с автомобилем'    
# используем подстроки для данных по недвижимости  
    if 'жиль' in index or 'недвиж' in index:
        return 'операции с недвижимостью'
# используем подстроки для данных по свальбе
    if 'свадьб' in index:
         return 'проведение свадьбы'
# используем подстроки для данных по образованию         
    if 'образо' in index:
        return 'получение образования'

df['purpose_category'] = df['purpose'].apply(purpose_category) # создаем новый столбец датафрейма
print(df['purpose_category'].value_counts()) # вывели количетво в столбце purpose_category по категориям
df.drop(['purpose', 'total_income'], axis=1, inplace=True)
display(df.head(20)) # вывели на экран для проверки
print(df['purpose_category'].unique()) # проверили уникальные значения в столбце purpose_category

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,C,операции с недвижимостью


['операции с недвижимостью' 'операции с автомобилем'
 'получение образования' 'проведение свадьбы']


### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

Есть ли зависимость между семейным положением и возвратом кредита в срок?

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Как разные цели кредита влияют на его возврат в срок?

##### Вывод 1:

К группе людей которые не возвращают кредит в срок, относятся люди с четыремя детьми, меньше количество просрочек у людей с одним ребенком.

Люди с семейным положение вдовец/вдова имееют большое количество не возврата кредита в срок, а с семейным положением женат/замужем имеют меньшее количество не возврата в срок.

Люди с уровнем дохода категории А и Е имееют большое количество не возврата кредита в срок, а люди с доходом категории В и D имеют меньшее соотношение невозвратов.   

Люди берущие кредит на оброзование, чаще не платят кредит в срок, а самый маленькое количество не возврата кредита в срок, взятых на операций с недвижимостью.

## Общий вывод:

Из графиков ниже мы видим что наиболее высоким уровнем просрочки относятся следующие типы людей: вдовец/вдова, люди с четыремя детьми,доход категории А и Е, цель кредита оброзование.

In [10]:
def age_category (age):
    if 19 <= age <= 24:
        return '19-24'
    if 25 <= age <= 30:
        return '25-30'
    if 31 <= age <= 36:
        return '31-36'
    if 37 <= age <= 44:
        return '37-43'
    if 40 <= age <= 49:
        return '44-49'
    if 50 <= age <= 59:
        return '50-55'
    if 60 <= age <= 65:
        return '56-65'
    if 66 <= age <= 75:
        return '65-75'

df['age_category'] = df['dob_years'].apply(age_category)
print(df['age_category'].value_counts())
display(df.head())

50-55    4657
37-43    4570
31-36    3420
25-30    2842
44-49    2489
56-65    1798
19-24     875
65-75     702
Name: age_category, dtype: int64


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income_category,purpose_category,age_category
0,1,8437,42,0,0,F,сотрудник,0,B,операции с недвижимостью,37-43
1,1,4024,36,1,0,F,сотрудник,0,C,операции с автомобилем,31-36
2,0,5623,33,1,0,M,сотрудник,0,C,операции с недвижимостью,31-36
3,3,4124,32,1,0,M,сотрудник,0,B,получение образования,31-36
4,0,340266,53,1,1,F,пенсионер,0,C,проведение свадьбы,50-55


In [11]:
total_table = df.pivot_table(index =['age_category'], columns =['children'], values = 'debt', aggfunc = 'mean')*100
display(total_table)
children = total_table.max()
print(children)

children,0,1,2,3,4,5
age_category,,,,,,
19-24,9.969325,10.052910,12.500000,0.000000,0.0,NaN
25-30,11.304348,10.894495,11.163895,6.122449,20.0,NaN
31-36,9.534050,10.086957,10.192837,11.290323,5.0,0.0
37-43,8.068783,9.004739,8.563135,6.504065,10.0,0.0
44-49,7.189901,7.099391,8.000000,10.000000,0.0,NaN
50-55,6.425406,7.429719,7.058824,0.000000,100.0,0.0
56-65,4.996990,3.968254,0.000000,0.000000,NaN,NaN
65-75,4.947526,5.882353,0.000000,NaN,NaN,NaN


children
0     11.304348
1     10.894495
2     12.500000
3     11.290323
4    100.000000
5      0.000000
dtype: float64


In [12]:
total_table = df.pivot_table(index =['age_category'], columns =['family_status_id'], values = 'debt', aggfunc = 'mean')*100
display(total_table)
family_status_id = total_table.max()
print(family_status_id)

family_status_id,0,1,2,3,4
age_category,,,,,
19-24,8.695652,9.545455,0.000000,12.500000,11.325967
25-30,10.737813,11.150442,16.666667,14.851485,11.196319
31-36,9.296838,11.095306,11.111111,5.405405,12.268519
37-43,7.823613,10.199790,4.109589,6.737589,9.955752
44-49,6.496986,8.686869,10.256410,6.111111,8.641975
50-55,6.080347,8.229426,5.585106,6.270627,7.506053
56-65,4.361371,4.467354,7.224335,6.034483,4.242424
65-75,4.624277,4.854369,5.755396,10.256410,2.666667


family_status_id
0    10.737813
1    11.150442
2    16.666667
3    14.851485
4    12.268519
dtype: float64


In [13]:
total_table = df.pivot_table(index =['age_category'], columns =['total_income_category'], values = 'debt', aggfunc = 'mean')*100
display(total_table)
total_income_category = total_table.max()
print(total_income_category)

total_income_category,A,B,C,D,E
age_category,,,,,
19-24,NaN,8.474576,10.386152,0.000000,NaN
25-30,NaN,8.805970,11.833489,9.090909,0.0
31-36,0.0,7.874865,10.672098,9.090909,NaN
37-43,10.0,6.356968,9.174032,8.000000,0.0
44-49,100.0,6.794682,7.488739,0.000000,0.0
50-55,0.0,7.414830,6.325897,5.825243,12.5
56-65,0.0,3.470032,5.140845,5.454545,25.0
65-75,NaN,4.938272,4.991394,5.555556,0.0


total_income_category
A    100.000000
B      8.805970
C     11.833489
D      9.090909
E     25.000000
dtype: float64


In [14]:
total_table = df.pivot_table(index =['age_category'], columns =['purpose_category'], values = 'debt', aggfunc = 'mean')*100
display(total_table)
purpose_category = total_table.max()
print(purpose_category)

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
age_category,,,,
19-24,12.422360,8.988764,9.433962,11.818182
25-30,12.190476,10.033898,13.928571,8.865248
31-36,12.916667,8.495575,11.038961,8.483290
37-43,10.064935,7.478261,8.782201,8.943089
44-49,6.653226,6.718750,9.070295,7.720588
50-55,6.951872,5.902481,7.449210,7.228916
56-65,6.648936,3.811659,5.952381,4.639175
65-75,4.761905,5.014749,5.594406,4.109589


purpose_category
операции с автомобилем      12.916667
операции с недвижимостью    10.033898
получение образования       13.928571
проведение свадьбы          11.818182
dtype: float64


In [16]:
total_table1 = df.pivot_table(index =['total_income_category', 'family_status_id', 'children'], columns =['age_category'], values = 'debt', aggfunc = 'mean')*100
display(total_table1.head())

age_category                                     19-24  25-30  31-36  37-43  \
total_income_category family_status_id children                               
A                     0                0           NaN    NaN    0.0    0.0   
                                       1           NaN    NaN    NaN   50.0   
                                       2           NaN    NaN    NaN    0.0   
                                       3           NaN    NaN    0.0    NaN   
                      1                0           NaN    NaN    0.0    0.0   

age_category                                     44-49  50-55  56-65  65-75  
total_income_category family_status_id children                              
A                     0                0           NaN    0.0    NaN    NaN  
                                       1           NaN    0.0    NaN    NaN  
                                       2           NaN    0.0    NaN    NaN  
                                       3           NaN    NaN    NaN    NaN  
                      1                0         100.0    NaN    NaN    NaN